In [1]:
import pandas as pd
import csv
import os
import requests
import urllib3
import csv
import time
import re
import numpy as np
# from random import randint
# from pandas import json_normalize
import censusgeocode as cg
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import shape
from datetime import datetime
os.chdir('C:/Users/jhuo/Documents/20220421_WABE_SB202/')

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
traveltimemaster5 = pd.read_csv('20220530_voter_data_all_vars.csv')

In [4]:
def race_recode(x):
    if x == 'WH':
        return(1)
    if x == ('U'):
        return(0)
    if x == ('OT'):
        return(0)
    else:
        return (-1)

In [5]:
traveltimemaster5['white'] = [race_recode(x) for x in traveltimemaster5.RACE]

In [6]:
traveltimemaster5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7528561 entries, 0 to 7528560
Data columns (total 41 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Unnamed: 0               int64  
 1   Unnamed: 0_x             int64  
 2   REGISTRATION_NUMBER      int64  
 3   BIRTHDATE_x              int64  
 4   RACE                     object 
 5   st_address               object 
 6   RESIDENCE_CITY           object 
 7   RESIDENCE_STATE          object 
 8   ZIP5                     int64  
 9   lat                      float64
 10  long                     float64
 11  min_driving_2020         float64
 12  min_driving_2022         float64
 13  driving_band_diff        float64
 14  BIRTHDATE_y              int64  
 15  min_transit_2020         float64
 16  min_transit_2022         float64
 17  transit_band_diff        float64
 18  urban                    bool   
 19  TRACTCE                  float64
 20  GEOID_left               float64
 21  NAME    

In [7]:
print('number of voters with 20 minute access who see their time go up')
len(traveltimemaster5[(traveltimemaster5.min_travel_time_2020 <=1200) &
                 (traveltimemaster5.min_travel_time_2022 <=1200) &
                 (traveltimemaster5.travel_diff < 0) ])

number of voters with 20 minute access who see their time go up


1856574

In [8]:
loseaccess = traveltimemaster5[(traveltimemaster5.min_travel_time_2020 <=1200) &
                 (traveltimemaster5.min_travel_time_2022 <=1200) &
                 (traveltimemaster5.travel_diff < 0) ]

In [9]:
len(loseaccess[loseaccess.urban == True])/len(loseaccess)

0.9325267939764319

In [10]:
len(loseaccess)/len(traveltimemaster5)

0.2466040987115599

In [11]:
len(traveltimemaster5[traveltimemaster5.urban == True])/len(traveltimemaster5)

0.7420277261484631

making crosstabs

In [12]:
def crosstab_traveltime(var_list):
    crosstab2020pct = pd.DataFrame(traveltimemaster5.groupby(var_list).min_travel_2020_recode.value_counts()/traveltimemaster5.groupby(var_list).REGISTRATION_NUMBER.count())
    crosstab2020pct.columns = ['pct_of_group']
    crosstab2020pct = crosstab2020pct.reset_index()

    crosstab2020cnt = pd.DataFrame(traveltimemaster5.groupby(var_list).min_travel_2020_recode.value_counts())
    crosstab2020cnt.columns = ['voter_count']
    crosstab2020cnt = crosstab2020cnt.reset_index()
    
    var_list.append('min_travel_2020_recode')
    newvarlist2020 = var_list.copy()
    var_list.remove('min_travel_2020_recode')
    crosstab2020merge = pd.merge(crosstab2020pct,crosstab2020cnt, on = newvarlist2020, how = 'outer')
    crosstab2020merge = crosstab2020merge.rename(columns = {'min_travel_2020_recode':'travel_time',
                                                           'pct_of_group':'pct_of_group_2020',
                                                           'voter_count':'voter_count_2020'})
    
    crosstab2022pct = pd.DataFrame(traveltimemaster5.groupby(var_list).min_travel_2022_recode.value_counts()/traveltimemaster5.groupby(var_list).REGISTRATION_NUMBER.count())
    crosstab2022pct.columns = ['pct_of_group']
    crosstab2022pct = crosstab2022pct.reset_index()

    crosstab2022cnt = pd.DataFrame(traveltimemaster5.groupby(var_list).min_travel_2022_recode.value_counts())
    crosstab2022cnt.columns = ['voter_count']
    crosstab2022cnt = crosstab2022cnt.reset_index()

    var_list.append('min_travel_2022_recode')
    newvarlist2022 = var_list.copy()
    var_list.remove('min_travel_2022_recode')
    crosstab2022merge = pd.merge(crosstab2022pct,crosstab2022cnt, on = newvarlist2022, how = 'outer')
    crosstab2022merge = crosstab2022merge.rename(columns = {'min_travel_2022_recode':'travel_time',
                                                           'pct_of_group':'pct_of_group_2022',
                                                           'voter_count':'voter_count_2022'})
    
    var_list.append('travel_time')
    mergevarlist = var_list.copy()
    crosstabmerge = pd.merge(crosstab2020merge, crosstab2022merge, on = mergevarlist, how = 'outer')
    crosstabmerge['pct_diff'] = crosstabmerge.pct_of_group_2020 - crosstabmerge.pct_of_group_2022
    return crosstabmerge

In [13]:
whitenonwhite = crosstab_traveltime(['white'])

In [14]:
whitenonwhite

,white,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,-1,600.0,0.737159,2018862,0.472808,1294882,0.264351
1,-1,1200.0,0.180447,494190,0.480717,1316541,-0.300270
2,-1,1800.0,0.020977,57451,0.043475,119066,-0.022498
3,0,600.0,0.628945,537557,0.401813,343428,0.227132
4,0,1200.0,0.254534,217549,0.517522,442324,-0.262988
5,0,1800.0,0.042096,35979,0.071519,61127,-0.029423
6,1,600.0,0.501028,1971624,0.357161,1405484,0.143867
7,1,1200.0,0.328276,1291819,0.532065,2093762,-0.203789
8,1,1800.0,0.060289,237245,0.096963,381563,-0.036674


In [15]:
0.143867/0.501028

0.28714363269118687

In [16]:
0.264351/0.737159 	

0.35860784444061594

In [17]:
urbanwhitenonwhite = crosstab_traveltime(['white','urban'])

In [18]:
urbanwhitenonwhite.sort_values(['urban','white','travel_time'])

,white,urban,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
1,-1,False,600.0,0.271472,91003,0.275828,92463,-0.004355
0,-1,False,1200.0,0.423077,141824,0.564590,189262,-0.141513
2,-1,False,1800.0,0.073632,24683,0.146414,49081,-0.072782
7,0,False,600.0,0.213910,36444,0.201108,34263,0.012801
6,0,False,1200.0,0.488293,83191,0.585733,99792,-0.097440
8,0,False,1800.0,0.107870,18378,0.184709,31469,-0.076838
13,1,False,600.0,0.207417,297969,0.208645,299733,-0.001228
12,1,False,1200.0,0.492369,707322,0.587485,843963,-0.095116
14,1,False,1800.0,0.111165,159696,0.177826,255459,-0.066661
3,-1,True,600.0,0.802110,1927859,0.500281,1202419,0.301828


In [19]:
urbanct = crosstab_traveltime(['urban'])

In [20]:
urbanct.sort_values(['urban','travel_time'])

,urban,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
1,False,600.0,0.219043,425416,0.219580,426459,-0.000537
0,False,1200.0,0.480052,932337,0.583380,1133017,-0.103328
2,False,1800.0,0.104398,202757,0.173008,336009,-0.068610
3,True,600.0,0.734395,4102627,0.468519,2617335,0.265876
4,True,1200.0,0.191755,1071221,0.486827,2719610,-0.295072
5,True,1800.0,0.022898,127918,0.040410,225747,-0.017512


In [21]:
0.265876/0.734395 	

0.36203405524275084

In [22]:
nocarct = crosstab_traveltime(['no_car'])

In [23]:
nocarct

,no_car,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,0,600.0,0.570286,3863411,0.386832,2620602,0.183453
1,0,1200.0,0.289060,1958242,0.521194,3530839,-0.232134
2,0,1800.0,0.048373,327701,0.081640,553072,-0.033267
3,1,600.0,0.881425,664632,0.561231,423192,0.320194
4,1,1200.0,0.060097,45316,0.426750,321788,-0.366653
5,1,1800.0,0.003944,2974,0.011517,8684,-0.007573


In [24]:
incomect = crosstab_traveltime(['income_qt'])

In [25]:
incomect

,income_qt,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,1.0,600.0,0.682120,1279428,0.534047,1001693,0.148073
1,1.0,1200.0,0.168115,315328,0.406837,763089,-0.238721
2,1.0,1800.0,0.029472,55279,0.055194,103526,-0.025723
3,2.0,600.0,0.537289,1008417,0.392211,736127,0.145077
4,2.0,1200.0,0.303073,568827,0.504528,946930,-0.201455
5,2.0,1800.0,0.054099,101537,0.093069,174677,-0.038969
6,3.0,600.0,0.569848,1069106,0.341390,640491,0.228457
7,3.0,1200.0,0.313931,588975,0.579556,1087320,-0.265624
8,3.0,1800.0,0.042416,79578,0.072004,135088,-0.029588
9,4.0,600.0,0.616292,1154666,0.350552,656784,0.265740


In [26]:
agect = crosstab_traveltime(['elderly'])

In [27]:
agect

,elderly,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,0,600.0,0.611591,3637822,0.405037,2409211,0.206554
1,0,1200.0,0.261498,1555422,0.514163,3058312,-0.252666
2,0,1800.0,0.041743,248291,0.072076,428716,-0.030333
3,1,600.0,0.563278,890221,0.401526,634583,0.161752
4,1,1200.0,0.283553,448136,0.502595,794315,-0.219041
5,1,1800.0,0.052128,82384,0.084180,133040,-0.032052


In [28]:
partyct = crosstab_traveltime(['party'])

In [29]:
partyct.sort_values(['party','travel_time'])

,party,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,D,600.0,0.839271,3062087,0.498253,1817880,0.341018
1,D,1200.0,0.122513,446991,0.474520,1731289,-0.352006
2,D,1800.0,0.010866,39643,0.026578,96969,-0.015712
4,R,600.0,0.377712,1462776,0.316206,1224577,0.061507
3,R,1200.0,0.400987,1552911,0.546311,2115713,-0.145325
5,R,1800.0,0.075140,290996,0.119964,464587,-0.044824
7,tie,600.0,0.443386,3164,0.185398,1323,0.257988
6,tie,1200.0,0.511771,3652,0.787416,5619,-0.275645
8,tie,1800.0,0.004484,32,0.027186,194,-0.022702


In [30]:
cityct = crosstab_traveltime(['RESIDENCE_CITY'])

In [31]:
countyct = crosstab_traveltime(['county_fips'])

In [32]:
racect = crosstab_traveltime(['RACE'])

In [33]:
racect

,RACE,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,AI,600.0,0.651414,18308,0.409180,11500,0.242234
1,AI,1200.0,0.238427,6701,0.517559,14546,-0.279132
2,AI,1800.0,0.033802,950,0.063903,1796,-0.030101
3,AP,600.0,0.719363,144874,0.405622,81689,0.313741
4,AP,1200.0,0.208901,42071,0.537216,108191,-0.328315
5,AP,1800.0,0.035031,7055,0.053259,10726,-0.018228
6,BH,600.0,0.747011,1662342,0.486874,1083454,0.260136
7,BH,1200.0,0.169963,378222,0.469350,1044456,-0.299387
8,BH,1800.0,0.018197,40495,0.041345,92005,-0.023147
9,HP,600.0,0.681051,193338,0.416508,118239,0.264543


In [34]:
def correlate(x,y):
    df = pd.DataFrame(traveltimemaster5.groupby(x)[y].value_counts()/traveltimemaster5.groupby(x).REGISTRATION_NUMBER.count())
    df.columns = ['pct_of_group']
    df = df.reset_index()
    countdf = pd.DataFrame(traveltimemaster5.groupby(x)[y].value_counts())
    countdf.columns = ['voter_count']
    countdf = countdf.reset_index()
    merge = pd.merge(df, countdf, on = [x,y], how = 'outer')
    return(merge.sort_values([x,y]))
    

In [35]:
correlate('urban','white')

,urban,white,pct_of_group,voter_count
1,False,-1,0.172602,335220
2,False,0,0.087722,170371
0,False,1,0.739676,1436569
4,True,-1,0.430239,2403485
5,True,0,0.122498,684325
3,True,1,0.447263,2498591


In [36]:
correlate('no_car','income_qt')

,no_car,income_qt,pct_of_group,voter_count
3,0,1.0,0.192336,1302983
2,0,2.0,0.263292,1783674
1,0,3.0,0.268359,1818006
0,0,4.0,0.273704,1854213
4,1,1.0,0.759481,572681
5,1,2.0,0.123586,93189
6,1,3.0,0.077078,58120
7,1,4.0,0.025670,19356


In [37]:
correlate('income_qt','no_car')

,income_qt,no_car,pct_of_group,voter_count
0,1.0,0,0.694678,1302983
1,1.0,1,0.305322,572681
2,2.0,0,0.950349,1783674
3,2.0,1,0.049651,93189
4,3.0,0,0.969021,1818006
5,3.0,1,0.030979,58120
6,4.0,0,0.989669,1854213
7,4.0,1,0.010331,19356


In [38]:
correlate('no_car','elderly')

,no_car,elderly,pct_of_group,voter_count
0,0,0,0.787791,5336903
1,0,1,0.212209,1437615
2,1,0,0.810602,611229
3,1,1,0.189398,142814


In [39]:
correlate('no_car','party').sort_values(['no_car','party'])

,no_car,party,pct_of_group,voter_count
1,0,D,0.447797,3033612
0,0,R,0.551473,3735963
2,0,tie,0.000701,4749
3,1,D,0.815464,614895
4,1,R,0.181370,136761
5,1,tie,0.003166,2387


In [40]:
nocarctycorr = correlate('no_car','county_fips').sort_values(['no_car','county_fips'])

In [41]:
nocarctycorr.to_csv('20220530_no_car_county.csv')

In [42]:
correlate('income_qt','elderly')

,income_qt,elderly,pct_of_group,voter_count
0,1.0,0,0.789079,1480047
1,1.0,1,0.210921,395617
2,2.0,0,0.785729,1474706
3,2.0,1,0.214271,402157
4,3.0,0,0.795956,1493314
5,3.0,1,0.204044,382812
6,4.0,0,0.789463,1479114
7,4.0,1,0.210537,394455


In [43]:
correlate('elderly','income_qt').sort_values(['elderly','income_qt'])

,elderly,income_qt,pct_of_group,voter_count
1,0,1.0,0.248826,1480047
3,0,2.0,0.247928,1474706
0,0,3.0,0.251056,1493314
2,0,4.0,0.248669,1479114
5,1,1.0,0.250323,395617
4,1,2.0,0.254461,402157
7,1,3.0,0.242220,382812
6,1,4.0,0.249587,394455


In [44]:
correlate('party','income_qt').sort_values(['party','income_qt'])

,party,income_qt,pct_of_group,voter_count
0,D,1.0,0.306590,1118596
2,D,2.0,0.221167,806928
1,D,3.0,0.261807,955204
3,D,4.0,0.205888,751185
7,R,1.0,0.194637,753776
5,R,2.0,0.276078,1069175
6,R,3.0,0.237094,918198
4,R,4.0,0.289717,1121994
8,tie,1.0,0.460482,3286
10,tie,2.0,0.103139,736


In [45]:
correlate('urban','no_car')

,urban,no_car,pct_of_group,voter_count
0,False,0,0.963842,1871935
1,False,1,0.036158,70225
2,True,0,0.877592,4902583
3,True,1,0.122408,683818


In [46]:
correlate('urban','income_qt')

,urban,income_qt,pct_of_group,voter_count
1,False,1.0,0.280502,544779
0,False,2.0,0.346397,672758
2,False,3.0,0.244055,473993
3,False,4.0,0.125654,244041
6,True,1.0,0.238237,1330885
7,True,2.0,0.215542,1204105
5,True,3.0,0.250990,1402133
4,True,4.0,0.291695,1629528


In [47]:
correlate('urban','elderly')

,urban,elderly,pct_of_group,voter_count
0,False,0,0.744142,1445243
1,False,1,0.255858,496917
2,True,0,0.806045,4502889
3,True,1,0.193955,1083512


In [48]:
correlate('urban','party')

,urban,party,pct_of_group,voter_count
1,False,D,0.106407,206659
0,False,R,0.892579,1733531
2,False,tie,0.000958,1861
3,True,D,0.616112,3441848
4,True,R,0.382929,2139193
5,True,tie,0.000944,5275


In [49]:
correlate('elderly','party')

,elderly,party,pct_of_group,voter_count
0,0,D,0.503090,2992444
1,0,R,0.495980,2950155
2,0,tie,0.000907,5394
4,1,D,0.415117,656063
3,1,R,0.583746,922569
5,1,tie,0.001102,1742


In [50]:
correlate('urban','RACE')

,urban,RACE,pct_of_group,voter_count
6,False,AI,0.002858,5551
5,False,AP,0.007642,14842
1,False,BH,0.142369,276503
3,False,HP,0.019733,38324
4,False,OT,0.010457,20310
2,False,U,0.077265,150061
0,False,WH,0.739676,1436569
13,True,AI,0.004037,22554
11,True,AP,0.033394,186550
8,True,BH,0.348851,1948823


In [51]:
correlate('no_car','RACE')

,no_car,RACE,pct_of_group,voter_count
6,0,AI,0.003696,25041
4,0,AP,0.027954,189375
1,0,BH,0.265948,1801669
3,0,HP,0.038627,261677
5,0,OT,0.019472,131915
2,0,U,0.094361,639251
0,0,WH,0.549942,3725590
13,1,AI,0.004063,3064
12,1,AP,0.015937,12017
7,1,BH,0.561847,423657


In [52]:
correlate('income_qt','RACE')

,income_qt,RACE,pct_of_group,voter_count
6,1.0,AI,0.003532,6624
5,1.0,AP,0.011775,22086
0,1.0,BH,0.453574,850752
3,1.0,HP,0.031714,59484
4,1.0,OT,0.016873,31649
2,1.0,U,0.082610,154949
1,1.0,WH,0.399922,750120
13,2.0,AI,0.003737,7014
12,2.0,AP,0.017424,32702
8,2.0,BH,0.298717,560650


In [53]:
correlate('elderly','RACE')

,elderly,RACE,pct_of_group,voter_count
6,0,AI,0.004379,26049
4,0,AP,0.028531,169708
1,0,BH,0.312891,1861116
3,0,HP,0.043956,261456
5,0,OT,0.022123,131593
2,0,U,0.098549,586185
0,0,WH,0.489570,2912025
13,1,AI,0.001301,2056
10,1,AP,0.020048,31684
8,1,BH,0.230450,364210


In [54]:
correlate('party','RACE')

,party,RACE,pct_of_group,voter_count
6,D,AI,0.004260,15543
4,D,AP,0.034656,126441
0,D,BH,0.473771,1728555
3,D,HP,0.043791,159770
5,D,OT,0.025482,92971
2,D,U,0.099785,364067
1,D,WH,0.318256,1161160
13,R,AI,0.003237,12537
11,R,AP,0.019298,74734
8,R,BH,0.127661,494397


In [55]:
urbanincome = crosstab_traveltime(['RACE','income_qt'])

In [56]:
urbanincome.sort_values(['RACE','income_qt','travel_time'])

,RACE,income_qt,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,AI,1.0,600.0,0.755435,5004,0.536081,3551,0.219354
1,AI,1.0,1200.0,0.146286,969,0.416214,2757,-0.269928
2,AI,1.0,1800.0,0.023098,153,0.044837,297,-0.021739
3,AI,2.0,600.0,0.591674,4150,0.413174,2898,0.178500
4,AI,2.0,1200.0,0.263473,1848,0.489877,3436,-0.226404
...,...,...,...,...,...,...,...,...
79,WH,3.0,1200.0,0.372422,370749,0.585711,583081,-0.213290
80,WH,3.0,1800.0,0.056042,55790,0.092657,92241,-0.036615
81,WH,4.0,600.0,0.592137,698279,0.339780,400686,0.252357
82,WH,4.0,1200.0,0.293539,346157,0.557388,657301,-0.263849


In [59]:
survey = pd.read_csv('MITU0031_OUTPUT2.tab', sep = '\t',low_memory=False)

In [ ]:
drop box vars - 
Q29 code 4, 
Q35 code 10
inputstate - ga is 13

In [71]:
surveyGA = survey[survey.inputstate == 13]

In [81]:
len(surveyGA[surveyGA.Q29 == 4])/len(surveyGA[surveyGA.Q28 != 99])

0.21476510067114093

In [82]:
survey[survey.Q29 == 4].inputstate.value_counts()

26    166
4     109
8     109
53     99
34     93
55     87
42     84
41     82
49     79
13     64
11     59
19     55
32     52
9      45
31     40
24     37
23     35
39     34
6      34
20     30
15     29
25     26
38     22
12     22
50     21
44     17
16     14
21     13
17     11
56     11
10      9
2       8
46      8
27      6
30      6
51      5
33      4
37      4
45      3
54      2
29      1
35      1
48      1
Name: inputstate, dtype: int64

In [79]:
surveyGA.Q28.value_counts()

99    702
1     259
2      35
9       4
Name: Q28, dtype: int64

In [75]:
len(surveyGA[surveyGA.Q35 == 10])/len(surveyGA)

0.011

In [74]:
600000/7000000

0.08571428571428572